In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.callbacks import *

In [ ]:
torch.cuda.is_available()

In [ ]:
np.random.seed(42)

In [ ]:
tfms=[flip_lr(p=0.5)],[]

In [ ]:
mixup=0.2

In [ ]:
loss_func = LabelSmoothingCrossEntropy()

In [ ]:
alpha=0.99
mom=0.9
eps=1e-6

In [ ]:
opt_func = partial(optim.Adam, betas=(mom,alpha), eps=eps)

In [ ]:
src = (ImageList.from_folder(path='train').split_by_rand_pct(0.2).label_from_folder())

In [ ]:
data = src.transform(tfms,size=(128)).databunch(bs=384).normalize().presize(256, scale=(0.35,1))

In [ ]:
learn = cnn_learner(data, models.densenet201,pretrained=False, metrics=[error_rate, accuracy, FBeta(beta=1,average='macro')],loss_func = loss_func,opt_func=opt_func,wd=1e-2)

In [ ]:
learn= learn.mixup(alpha=mixup)

In [ ]:
learn.bn_wd=False

In [ ]:
learn.true_wd=True

In [ ]:
learn = learn.to_fp16(dynamic=True)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr=2.5e-2

In [ ]:
learn.fit_one_cycle(80, lr, div_factor=10, pct_start=0.3)


In [ ]:
learn.save('dn209-exp-1-round-1')

In [ ]:
learn